In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
import time
from PID import PID

In [ ]:
# -------------------------
# Parameters
# -------------------------
camera_width = 320
camera_height = 240
forward_speed = 0.1
turn_speed = 0.1
turn_duration = 0.4  # seconds

# PID coefficients
steering_pid = PID(Kp=0.0001, Ki=0.0, Kd=0.0005, setpoint=camera_width / 2)

# HSV thresholds
lower_green = np.array([35, 60, 60])
upper_green = np.array([85, 255, 255])
lower_white = np.array([0, 0, 150])
upper_white = np.array([180, 60, 255])

# -------------------------
# Hardware
# -------------------------
camera = Camera.instance(width=camera_width, height=camera_height)
robot = Robot()

# -------------------------
# Widgets
# -------------------------
line_mask_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)

robot_stop = True
stop_button = widgets.Button(description="STOP", button_style='danger', layout=widgets.Layout(width='100px', height='50px'))
start_button = widgets.Button(description="START", button_style='success', layout=widgets.Layout(width='100px', height='50px'))

def stop_robot(b):
    global robot_stop
    robot_stop = True
    robot.stop()

def start_robot(b):
    global robot_stop
    robot_stop = False

stop_button.on_click(stop_robot)
start_button.on_click(start_robot)

display(widgets.VBox([
    line_mask_widget,
    widgets.HBox([start_button, stop_button])
]))

# -------------------------
# State Machine
# -------------------------
state = "FOLLOW_LINE"

def follow_black_line(frame):
    global state, robot_stop

    h, w, _ = frame.shape
    roi = frame  # full frame

    # Enhance contrast
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])

    # Color masks
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Adaptive threshold only
    mask_black = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 15, 5  # C=5 is safer
    )

    # Optional: morphological close to fill gaps
    kernel = np.ones((3,3), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)


    # Morphological cleanup
    kernel = np.ones((3,3), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)

    # -------------------------
    # Green-square turn detection
    # -------------------------
    contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    turn_signal = None

    for cnt in contours:
        x, y, gw, gh = cv2.boundingRect(cnt)
        cx = x + gw // 2
        cy = y + gh // 2

        # Region just below the square
        below_y1 = min(mask_white.shape[0]-1, y + gh)
        below_y2 = min(mask_white.shape[0], y + gh + 10)
        strip_below = mask_white[below_y1:below_y2, x:x+gw]
        white_below = np.mean(strip_below) > 128 if strip_below.size > 0 else False

        # Side regions (same vertical band)
        side_y1 = y
        side_y2 = y + gh
        left_x1 = max(0, x - gw)
        left_x2 = x
        right_x1 = x + gw
        right_x2 = min(mask_black.shape[1], x + 2 * gw)

        left_region = mask_black[side_y1:side_y2, left_x1:left_x2]
        right_region = mask_black[side_y1:side_y2, right_x1:right_x2]

        black_left = np.mean(left_region) > 128 if left_region.size > 0 else False
        black_right = np.mean(right_region) > 128 if right_region.size > 0 else False

        # Decision logic
        if white_below:
            if black_right and not black_left:
                turn_signal = "LEFT"
            elif black_left and not black_right:
                turn_signal = "RIGHT"

        # Debug visualization
        color = (0,255,0) if white_below else (0,0,255)
        cv2.rectangle(roi, (x, y), (x+gw, y+gh), color, 2)

    # -------------------------
    # Line centroid
    # -------------------------
    M = cv2.moments(mask_black)
    line_found = M["m00"] > 0
    if line_found:
        cx_line = int(M["m10"] / M["m00"])
        cv2.circle(roi, (cx_line, h//2), 5, (0,0,255), -1)
        steering_pid.setpoint = w / 2

    # -------------------------
    # State machine
    # -------------------------
    if state == "FOLLOW_LINE":
        if line_found:
            steering = steering_pid.update(cx_line)
            left_motor = forward_speed + steering
            right_motor = forward_speed - steering
        else:
            left_motor = right_motor = 0

        if turn_signal == "LEFT":
            state = "TURN_LEFT"
        elif turn_signal == "RIGHT":
            state = "TURN_RIGHT"

    elif state == "TURN_LEFT":
        robot.set_motors(-turn_speed, turn_speed)
        time.sleep(turn_duration)
        state = "FOLLOW_LINE"
        return

    elif state == "TURN_RIGHT":
        robot.set_motors(turn_speed, -turn_speed)
        time.sleep(turn_duration)
        state = "FOLLOW_LINE"
        return

    # Safety stop
    if robot_stop:
        left_motor = right_motor = 0

    robot.set_motors(left_motor, right_motor)

    # Debug overlay
    debug_vis = cv2.addWeighted(roi, 0.8, cv2.cvtColor(mask_black, cv2.COLOR_GRAY2BGR), 0.3, 0)
    line_mask_widget.value = bgr8_to_jpeg(debug_vis)

# Attach callback
camera.observe(lambda change: follow_black_line(change['new']), names='value')


In [ ]:
robot.stop()
camera.stop()